# Building Blocks of Language Models
In this notebook, we will explore:
- <b>Tokenization</b>: Breaking text into tokens
- <b>Embeddings</b>: Converting tokens to numerical vectors
- <b>Positional Encoding</b>: Adding position information
- <b>Self-Attention</b>: Using all of the above to understand relationships
- <b>Prediction</b>: Generating the next token using the model's output

## Tokenization
- The model can’t read words directly — it only understands numbers. So we need to convert text → tokens → numbers.
- tokenize() splits into sub-word pieces.
- encode() converts them into numeric IDs.
- decode() reverses the process.

In [ ]:
from transformers import AutoTokenizer

# Load a small open model's tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

text = "Small Language Models are powerful and efficient!"
tokens = tokenizer.tokenize(text)
ids = tokenizer.encode(text)

print("Original text:", text)
print("Tokens:", tokens)
print("Token IDs:", ids)
print("Decoded back:", tokenizer.decode(ids))

### A Simple Tokenizer Implementation (Toy Tokenizer)

In [ ]:
import re

def basic_tokenizer(text):
    # Lowercase + split on spaces & punctuation
    text = text.lower()
    tokens = re.findall(r"\w+|[^\w\s]", text)
    return tokens

sample_text = "Small Language Models are powerful and efficient!"
tokens = basic_tokenizer(sample_text)
print(tokens)
    

### Real GPT-2 BPE tokenizer
1. GPT-2 uses subword tokenization (Byte Pair Encoding, BPE) to handle rare words efficiently.
2. Tokens are not always whole words; BPE splits or merges based on frequency.
3. The tokenizer’s vocabulary is fixed, built before model training. 
   Tokenizer training is often done on a subset of the full corpus (e.g., 1–10% of data), if the corpus is extremely large, to save time.
   The subset must be representative of all text the model will encounter.
   The vocabulary generated is then fixed and used throughout model training.

### Analogy
- Think of the vocabulary as a dictionary for the model.
- You want the dictionary to cover all common words in the language you’re going to teach the model.
- Making a dictionary from Shakespeare when you’re training on Wikipedia would work poorly — it won’t reflect the frequency of words in Wikipedia.

In [ ]:
from transformers import AutoTokenizer

# Load GPT-2 tokenizer (uses real BPE)
hf_tokenizer = AutoTokenizer.from_pretrained("gpt2")

text = "Small Language Models are powerful and efficient!"
tokens_hf = hf_tokenizer.tokenize(text)
ids_hf = hf_tokenizer.encode(text)

print("Original text:", text)
print("\nHugging Face BPE Tokens:", tokens_hf)
print("Token IDs:", ids_hf)
print("Decoded back:", hf_tokenizer.decode(ids_hf))
# Decoding some random token ID - Prints a word referred in the tokenizer vocabulary
print("Random Token:", hf_tokenizer.decode(18710))  

---

## Embeddings
### Definition of embedding:
* Numerical vector representing an object (word, token, or sentence) in high-dimensional space.
* Similar objects → vectors close together; dissimilar → far apart.

### Purpose in Transformers / LLMs:
* Converts symbolic text into numbers so the model can process it.
* Captures semantic meaning and, in last_hidden_state, contextual meaning.

#### Example:
- Text: "Small Language Models"
- Tokens: ["Small", "ĠLanguage", "ĠModels"]
- Each token → 768-dimensional vector (GPT-2 small).

### Why embeddings are important:
* Provide contextualization: combined with attention, the model “understands” meaning in context.
* Allow semantic similarity: similar words (e.g., “king” & “queen”) have similar vectors.
* Serve as the starting point for downstream tasks: next-token prediction, classification, etc.

### Analogy:
* Think of a map of cities
* Each city = token
* Coordinates = embedding vector
* Nearby cities = similar meaning
* Moving through the map = attention + feed-forward layers updating embeddings

### last_hidden_state:
* Tensor of shape [batch_size, seq_len, hidden_size]
* Each token has a contextualized embedding vector after passing through Transformer layers.
* Before passing through layers → embeddings are basic numeric representations of tokens.
* After layers → embeddings are contextualized, rich representations used for predictions.


In [ ]:
# Extract Embeddings from GPT-2
import torch
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")

text = "Small Language Models are powerful and efficient!"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state  # shape: [batch_size, seq_len, hidden_size]

print("Embedding shape:", embeddings.shape)
print("Number of tokens:", embeddings.shape[1])
print("Hidden size:", embeddings.shape[2])


In [ ]:
# Visualizing Embeddings with PCA
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

tokens = tokenizer.tokenize(text)
emb_np = embeddings[0].numpy()  # convert from tensor to numpy

# Reduce 768-d to 2D
pca = PCA(n_components=2)
emb_2d = pca.fit_transform(emb_np)

plt.figure(figsize=(8,4))
plt.scatter(emb_2d[:,0], emb_2d[:,1])
for i, token in enumerate(tokens):
    plt.annotate(token, (emb_2d[i,0], emb_2d[i,1]))
plt.title("2D PCA of GPT-2 Token Embeddings")
plt.show()


---

## Positional Encoding (Sinusoidal + Learned)
Transformers don’t process words sequentially like RNNs.
So they need to know where each token is in the sentence.

They add a positional vector (e.g., sine/cosine pattern or learned embedding)
so “The cat” ≠ “cat The”.

Result = (embedding + position) for each token.

### 1. Sinusoidal Positional Encoding
#### Implementation

In [ ]:
import torch
import math

def sinusoidal_positional_encoding(seq_len, d_model):
    """
    Create sinusoidal positional encodings of shape (seq_len, d_model)
    As described in 'Attention is All You Need'
    """
    pe = torch.zeros(seq_len, d_model)
    position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)

    # frequencies: 10000^(2i/d_model)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

    # apply sin to even indices, cos to odd
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    return pe


#### Visualize Sinusoidal Encoding Patterns

In [ ]:
import matplotlib.pyplot as plt

seq_len = 1024
d_model = 768

pe = sinusoidal_positional_encoding(seq_len, d_model)

plt.figure(figsize=(12, 6))
plt.imshow(pe, aspect='auto', cmap='viridis')
plt.colorbar()
plt.title("Sinusoidal Positional Encoding (1024 positions × 768 dims)")
plt.xlabel("Embedding Dimension")
plt.ylabel("Position Index")
plt.show()

#### Extract GPT-2 Token Embeddings for Comparison

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")

text = "The dog chased the ball."
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    token_emb = outputs.last_hidden_state[0]  # shape: (seq_len, d_model)

print("Token embedding shape:", token_emb.shape)


#### Add Sinusoidal Positional Encoding to Token Embeddings

In [ ]:
seq_len = token_emb.shape[0]
d_model = token_emb.shape[1]

pos_enc = sinusoidal_positional_encoding(seq_len, d_model)

# Add position encoding to token embeddings
final_emb = token_emb + pos_enc

print("Final embedding shape:", final_emb.shape)

### 2. Learned Positional Embeddings (GPT-style)
#### Implementation

In [ ]:
import torch.nn as nn

class LearnedPositionEmbedding(nn.Module):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.pos_emb = nn.Embedding(max_len, d_model)

    def forward(self, seq_len):
        positions = torch.arange(0, seq_len, dtype=torch.long)
        return self.pos_emb(positions)  # (seq_len, d_model)

max_len = 1024
d_model = 768

learned_pe = LearnedPositionEmbedding(max_len, d_model)
pos_learned = learned_pe(seq_len)

print("Learned positional embedding shape:", pos_learned.shape)


#### Combine Learned Positional Embeddings with Token Embeddings

In [ ]:
final_emb_learned = token_emb + pos_learned
print("Final embedding (learned PE) shape:", final_emb_learned.shape)

---

## Attention
### Attention Visualization (using a real GPT-2 model)

#### 📘 Model Context
- **Model:** `GPT-2` (decoder-only Transformer)
- **Architecture:** 12 layers × 12 attention heads  
- **Attention type:** *Causal self-attention* → each token can only attend to **previous** tokens (no future look-ahead)


#### 🔍 What the Heatmap Shows
- **Rows (Y-axis):** Query tokens — the ones *attending*  
- **Columns (X-axis):** Key tokens — the ones *being attended to*  
- **Bright cells:** High attention weight → strong focus/relationship  
- **Diagonal line:** Each token attends to itself (self-attention)  
- **Dark upper triangle:** Causal mask (future tokens are hidden)


#### 🧩 Data Shapes
- `attentions[layer]` → shape `[batch, heads, seq_len, seq_len]`  
  - Each entry = one attention matrix for a single head  
- `last_hidden_state` → contextualized token embeddings  
- `attentions` → relationships between tokens (who “looks” at whom)


#### 🎨 Visualization Settings
- Color map: `"magma"` for strong contrast  
- Log scale: `np.log1p(attention)` to highlight smaller differences  
- Color range: `vmin=0.0`, `vmax≈0.15–0.25` to make bright regions pop  
- Convert tensor before plotting:  
  ```python
  attention = attention.cpu().numpy()


In [ ]:
# Understanding & Visualizing Attention (GPT-2)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2", output_attentions=True)
model.eval()

sentence = "The dog chased the ball because it was excited."
inputs = tokenizer(sentence, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs, output_attentions=True)

attentions = outputs.attentions  # List of length L (num layers), each element is a tensor of shape (num_heads, seq_len, seq_len)
# So attentions[0] → first layer, all heads in that layer.
# attentions[1] → second layer, all heads in that layer, and so on.
# attentions[0].shape gives (batch_size, num_heads, seq_len, seq_len). [1] selects the second dimension, which corresponds to number of heads (num_heads).
print(f"Layers: {len(attentions)} | Heads per layer: {attentions[0].shape[1]}")

# Visualize one attention head from the last layer
layer = -1  # last layer
head = 0   # first attention head

attention = attentions[layer][0, head].cpu()
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

plt.figure(figsize=(8,6))
sns.heatmap(np.log1p(attention), xticklabels=tokens, yticklabels=tokens, cmap="magma", square=True, vmin=0.0, vmax=0.15)
plt.title(f"GPT-2 Attention Map (Layer {layer}, Head {head})")
plt.xlabel("Key Tokens")
plt.ylabel("Query Tokens")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Self-Attention Implementation (From Scratch)


#### 1. Set up
We will simulate a very small model with:
 - Sequence Length (T): 3 tokens
 - Hidden Size (D): 4 dimensions
 - Head Dimension (dk): 2 dimensions (We'll use a single head for simplicity).
 - We need three sets of Learned Weights (WQ, WK, WV), which the model learned during training. These weights transform the input embedding X into Q, K, and V.
 - The input X is the combined token and positional embedding.

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

# --- 1. Define Shapes and Input ---
# D_model (Hidden size) = 4
# d_k (Head dimension) = 2
# T (Sequence Length) = 3

D_MODEL = 4
D_HEAD = 2
SEQ_LEN = 3

# Simulate the input X (Token Embeddings + Positional Embeddings)
# Shape: [SEQ_LEN, D_MODEL] -> [3, 4]
X = torch.tensor([
    [1.0, 0.5, 0.2, 0.1],  # x1: "Small"
    [0.1, 1.2, 0.3, 0.5],  # x2: "Language"
    [0.8, 0.4, 1.1, 0.2]   # x3: "Models"
], dtype=torch.float32)

print("Input Embeddings (X) Shape:", X.shape)

# --- 2. Simulate Learned Weights ---
# Weights must transform D_MODEL -> D_HEAD
# Shape: [D_MODEL, D_HEAD] -> [4, 2]
W_Q = torch.randn(D_MODEL, D_HEAD)
W_K = torch.randn(D_MODEL, D_HEAD)
W_V = torch.randn(D_MODEL, D_HEAD)

print("Weight Matrix (W_Q) Shape:", W_Q.shape)

#### 2. Computing Q, K, and V
Q, K, and V are generated by simple matrix multiplication:
- Q = WQ . X
- K = WK . X
- V = WV . X

In [ ]:
# --- 3. Generate Q, K, V Matrices ---
# Shape: [SEQ_LEN, D_MODEL] * [D_MODEL, D_HEAD] = [SEQ_LEN, D_HEAD]
Q = X @ W_Q
K = X @ W_K
V = X @ W_V

print("\\nQ Matrix (Queries):\\n", Q)
print("K Matrix (Keys):\\n", K)
print("V Matrix (Values):\\n", V)
print(f"Q, K, V Shapes: {Q.shape}")

#### 3. Computing Attention Scores and Weights

This is the heart of the mechanism. The scores are computed by multiplying Q and the transpose of K (K^T).
1. Scores: Scores=Q⋅K^T
 - Shape: [3,2]⋅[2,3]=[3,3] (The T×T attention matrix!)

2. Weights: Weights=Softmax(Scores/sqroot(d_k))
 - The devision by sqrt(d_k) (scaling factor) is to normalize the scores and prevent the softmax from becoming too large.
 - The Softmax converts the scores into probabilities (attention weights).

In [ ]:
# --- 4. Compute Attention Scores (Q * K^T) ---
# Scale factor (square root of the head dimension)
scale_factor = torch.sqrt(torch.tensor(D_HEAD, dtype=torch.float32))

# Scores shape: [SEQ_LEN, SEQ_LEN] -> [3, 3]
Scores = Q @ K.T / scale_factor

print("\\nAttention Scores (Q · K^T / sqrt(d_k)):\\n", Scores)

# --- 5. Apply Causal Mask (for GPT-style attention) ---
# We use float('-inf') to mask (hide) future tokens.
# Tokens can only attend to tokens that came before them.
mask = torch.triu(torch.ones(SEQ_LEN, SEQ_LEN) * float('-inf'), diagonal=1)

# Apply mask to scores
Masked_Scores = Scores + mask

print("\\nMasked Attention Scores (Causal):\\n", Masked_Scores)

# --- 6. Compute Attention Weights (Softmax) ---
# Softmax ensures weights sum to 1 for each row (Query)
Attention_Weights = F.softmax(Masked_Scores, dim=-1)

print("\\nAttention Weights (Softmax):\\n", Attention_Weights)

# Check that rows sum to 1 (for first token, only one non-zero entry)
print("Row Sums (Should be 1):", torch.sum(Attention_Weights, dim=1))

#### 4. Computing the Attention Output

The final attention output is the weighted sum of the Value vectors. This is where the contextual information is merged.
 - Output=Attention Weights⋅V

In [ ]:
# --- 7. Compute Final Attention Output ---
# Output shape: [SEQ_LEN, SEQ_LEN] * [SEQ_LEN, D_HEAD] = [SEQ_LEN, D_HEAD]
Attention_Output = Attention_Weights @ V

print("\\nAttention Output (Weighted Sum of V):\\n", Attention_Output)
print("Attention Output Shape:", Attention_Output.shape)

# Example Interpretation:
# The first row of Attention_Output is the contextualized vector for "Small".
# Because of the causal mask, the first token can only attend to itself, 
# so the first row of Attention_Output should be close to the first row of V.

# Check (V[0] vs Output[0]):
print("\\nV[0] (for 'Small'):", V[0])
print("Output[0] ('Small' Contextualized):", Attention_Output[0])

#### 5. Add & Layer Normalization 1 (After MHA)

After the MHA block computes the contextualized output, we apply two crucial mechanisms that ensure the model can train deeply and effectively: the Residual Connection (Add) and Layer Normalization (Norm).

##### 5.1. The Residual Connection (Add)

This step takes the output from the Multi-Head Attention layer and simply **adds** it to the layer's original input.

$$\mathbf{Y}_{\text{res}} = X + \text{MHA}_{\text{Output}}$$

* **Purpose:** This creates a **shortcut** around the attention block, allowing the gradient (the learning signal) to flow directly through the network during training. Without this shortcut, gradients would vanish in deep models, preventing the first layers from learning. It ensures that the model can, at worst, maintain its previous state of knowledge if the attention layer provides no meaningful update.

##### 5.2. Layer Normalization (Norm)

Layer Normalization is applied immediately after the residual addition.

* **Purpose:** LayerNorm stabilizes the activation values. The MHA block involves intense matrix math (dot products and softmax) that can cause the resulting vector's numerical range to shift wildly.
* **Mechanism:** LayerNorm calculates the mean and variance across the entire **feature dimension** ($D_{\text{model}}$) for **each token individually**. It then centers the data ($\mu=0$) and scales it ($\sigma^2=1$).
* **Result:** This ensures that the input to the next major block (the Feed-Forward Network) is always consistent and within a stable range, which drastically speeds up training convergence.

**In essence, the "Add & Norm" sequence is about stability:** the **Add** stabilizes the *learning process*, and the **Norm** stabilizes the *data values*.

In [ ]:
# --- Continuing from the Attention Output (Attention_Output) ---
# X is the original input embedding tensor from the beginning of the layer

# 1. Residual Connection (ADD)
# Requires X to have the same shape as Attention_Output.
# NOTE: In a real model, the output projection makes Attention_Output return to D_MODEL (4 in our case).
# We simulate this by adjusting the shape of Attention_Output from [3, 2] to [3, 4] 
# and then adding it back to X [3, 4].

# Let's re-use X (Input Embeddings) from earlier for the residual path:
# X shape: [3, 4]
# We must first project Attention_Output [3, 2] back to [3, 4] to add it to X.
# We simulate the Output Projection W_O here for simplicity.

W_O_sim = torch.randn(D_HEAD, D_MODEL) # [2, 4]
MHA_Output = Attention_Output @ W_O_sim # [3, 2] @ [2, 4] = [3, 4]

# Add & Norm 1
Y_res_1 = X + MHA_Output 
print("Residual Output 1 Shape (X + MHA_Output):", Y_res_1.shape)

# 2. Layer Normalization
# Normalizes across the feature dimension (dim=1, D_MODEL=4)
ln1 = torch.nn.LayerNorm(D_MODEL) 
Output_LN1 = ln1(Y_res_1)

print("LayerNorm 1 Output (Input to FFN) Shape:", Output_LN1.shape)

#### 6. The Feed-Forward Network (FFN)

The Feed-Forward Network is the second major sub-layer in the Transformer block, positioned right after the first Add & Layer Normalization.

##### 6.1. Conceptual Role

While the Multi-Head Attention (MHA) block is responsible for **mixing information *across* the entire sequence** (contextualizing a token based on others), the FFN works **independently on each token's embedding** to deepen its feature representation.

* **Non-Linearity:** The FFN applies critical non-linear transformations (using the $\text{GELU}$ activation function). This is essential because the world is complex, and the model needs non-linear functions to learn intricate relationships and patterns.
* **Feature Expansion:** The FFN expands the token's feature vector to a higher dimension (e.g., $D_{\text{model}} \rightarrow 4 \times D_{\text{model}}$) and then contracts it back down. This expansion-contraction forces the model to learn a richer, concentrated set of features for each token.

##### 6.2. The Two-Layer Structure

The FFN is a standard two-layer Multi-Layer Perceptron (MLP):

1.  **Expansion Layer ($\mathbf{W}_{\text{in}}$):** Takes the normalized input vector (e.g., 768-dim) and expands it fourfold (e.g., to 3072-dim). An activation function ($\text{GELU}$) is applied here to introduce non-linearity.
2.  **Contraction Layer ($\mathbf{W}_{\text{out}}$):** Takes the expanded vector (e.g., 3072-dim) and projects it back down to the original hidden size (e.g., 768-dim).

$$\text{FFN}(\mathbf{x}) = \mathbf{x} \cdot \mathbf{W}_{\text{in}} \xrightarrow{\text{GELU}} \mathbf{y}' \cdot \mathbf{W}_{\text{out}}$$

In [ ]:
# --- Define FFN Dimensions ---
# D_MODEL was set to 4 in our simulation
D_MODEL = 4
D_FFN = D_MODEL * 4 # Standard expansion factor of 4
print(f"FFN will expand from {D_MODEL} to {D_FFN} and back to {D_MODEL}")

# --- FFN Implementation Class ---
class SimpleFFN(torch.nn.Module):
    """
    A simple Feed-Forward Network (FFN) with expansion and contraction layers.
    """
    def __init__(self, d_model, d_ffn):
        super().__init__()
        # 1. Expansion layer: D_MODEL -> D_FFN
        self.linear_in = torch.nn.Linear(d_model, d_ffn)
        # 2. Contraction layer: D_FFN -> D_MODEL
        self.linear_out = torch.nn.Linear(d_ffn, d_model)
        # Non-linearity (GPT models typically use GELU)
        self.activation = torch.nn.GELU() 

    def forward(self, x):
        # Apply expansion and activation
        x = self.linear_in(x)
        x = self.activation(x)
        # Apply contraction
        x = self.linear_out(x)
        return x

# Assuming 'Output_LN1' is the tensor produced after the first Layer Norm (Add & Norm 1)
# Re-define Output_LN1 for execution context, if the previous code cells were not run
# Since we cannot run the previous cells, we'll create a dummy tensor with the expected shape [3, 4]

# --- IMPORTANT: Ensure Output_LN1 exists from previous run or define it ---
# You should ensure your previous cells run up to the first Layer Norm.
# For demonstration purposes here, we'll use a placeholder variable:
try:
    # This assumes Output_LN1 was generated successfully in the previous cell block
    print(f"Input to FFN (Output_LN1) successfully loaded with shape: {Output_LN1.shape}")
except NameError:
    # If the variable is not defined (e.g., running this cell isolated), use a placeholder:
    print("WARNING: Output_LN1 not found. Creating placeholder tensor for FFN input.")
    Output_LN1 = torch.randn(3, 4) 

# --- 1. Run the FFN ---
ffn = SimpleFFN(D_MODEL, D_FFN)
FFN_Output = ffn(Output_LN1)

print("\\n--- FFN Execution ---")
print("FFN Output Shape:", FFN_Output.shape)

#### 7. Add & Layer Normalization 2 (The Final Steps)

After the FFN, the output is passed through the same stabilization steps again:

1.  **Residual Connection (Add):** The FFN output is added to the FFN's input ($\text{Output}_{\text{LN1}}$).
2.  **Layer Normalization (Norm):** The result is normalized.

This final `Add & Norm` ensures that the resulting contextualized vector is stable and ready to be passed to the next Transformer layer in the stack.

**Key Takeaway:** The FFN works on *what* a token is, while the MHA works on *where* a token is in context. Together, they create a powerful, context-aware representation.

In [ ]:
# --- 2. Add & Layer Norm 2 (The Final Steps) ---

# 2.1. Residual Connection (ADD)
# The FFN output is added to its input (Output_LN1)
Y_res_2 = Output_LN1 + FFN_Output

print("Residual Output 2 Shape (Output_LN1 + FFN_Output):", Y_res_2.shape)


# 2.2. Layer Normalization
ln2 = torch.nn.LayerNorm(D_MODEL) 
Final_Layer_Output = ln2(Y_res_2)

print("Final Layer Output Shape:", Final_Layer_Output.shape)
print("--- Layer Complete ---")

## Prediction - Logits and Next Token Generation

The ultimate function of a Generative Language Model (LLM) is to predict the **next token** in a sequence. This prediction is based on the final, contextualized embedding vector produced by the very last Transformer layer.

### 1. The Final Transformation: Logits

The prediction is not a word itself, but a set of raw numerical scores called **Logits**.

1.  **Logits Projection (The LM Head):** The final contextualized embedding, which is $D_{\text{model}}$-dimensional (e.g., 768), is projected onto the size of the model's entire vocabulary (e.g., 50,257 for GPT-2). This is done via a final, linear layer known as the **Language Modeling Head**.
2.  **Output:** The Logits tensor has a shape of $[1, \text{Vocab Size}]$. Each number in this tensor represents the raw, unnormalized score for how likely the corresponding token ID is to be the next token.

### 2 Softmax and Sampling

The Logits must be converted into a usable prediction:

1.  **Softmax:** The $\text{Softmax}$ function is applied to the Logits to transform them into a probability distribution. The resulting probabilities sum up to 1, showing the confidence score for every possible next token.
2.  **Sampling:** The model then uses these probabilities to choose the next token ID.
    * **Greedy Decoding:** Choosing the token with the absolute highest probability (the `torch.argmax` method).
    * **Sampling:** Choosing a token based on the probabilities (e.g., if "the" has 80% and "a" has 10%, the model might randomly pick "the" 8 out of 10 times).

The final loop of the Transformer is:
$$\text{Contextualized Vector} \xrightarrow{\text{LM Head}} \text{Logits} \xrightarrow{\text{Softmax}} \text{Probabilities} \xrightarrow{\text{Sampling}} \text{New Token}$$

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- Setup ---
tokenizer = AutoTokenizer.from_pretrained("gpt2")
# AutoModelForCausalLM includes the final prediction layer (LM Head)
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.eval()

# Input sentence for the model to continue
sentence = "The dog chased the ball because it"
print("Input:", sentence)

inputs = tokenizer(sentence, return_tensors="pt")
input_ids = inputs["input_ids"]

# --- 1. Forward Pass to get Logits ---
with torch.no_grad():
    outputs = model(**inputs)
    # The output object for AutoModelForCausalLM contains the logits
    logits = outputs.logits

print("\\nLogits Shape (Batch, Seq_Len, Vocab_Size):", logits.shape)

# --- 2. Focus on the LAST Token's Logits ---
# Logits for the last token are used to predict the next word.
last_token_logits = logits[0, -1, :] 

print("Logits for next token prediction Shape:", last_token_logits.shape)

# --- 3. Convert Logits to Probabilities (Softmax) and Sample ---

# Apply softmax to convert raw scores into a probability distribution
probabilities = F.softmax(last_token_logits, dim=-1)

# Find the token with the highest probability (Greedy Decoding)
next_token_id_greedy = torch.argmax(probabilities).item()

# --- 4. Decode and Display Prediction ---
next_token_greedy = tokenizer.decode(next_token_id_greedy)

print("-" * 50)
print(f"Top Logit Value: {last_token_logits.max().item():.2f}")
print(f"Greedy Prediction (Highest Probability): '{next_token_greedy}'")
print(f"Full Sequence: '{sentence}{next_token_greedy}'")
print("-" * 50)

# --- Bonus: Show Top 5 Predictions ---
top_5_probs, top_5_indices = torch.topk(probabilities, 5)

print("\\nTop 5 Next Token Predictions:")
for i in range(5):
    token = tokenizer.decode(top_5_indices[i].item())
    prob = top_5_probs[i].item()
    print(f"  {i+1}. Token: '{token}' | Probability: {prob*100:.2f}%")

Input: The dog chased the ball because it
\nLogits Shape (Batch, Seq_Len, Vocab_Size): torch.Size([1, 7, 50257])
Logits for next token prediction Shape: torch.Size([50257])
--------------------------------------------------
Top Logit Value: -115.58
Greedy Prediction (Highest Probability): ' was'
Full Sequence: 'The dog chased the ball because it was'
--------------------------------------------------
\nTop 5 Next Token Predictions:
  1. Token: ' was' | Probability: 42.30%
  2. Token: ' had' | Probability: 8.79%
  3. Token: ' didn' | Probability: 3.87%
  4. Token: ' wanted' | Probability: 2.91%
  5. Token: ' wasn' | Probability: 2.67%
  6. Token: ' could' | Probability: 1.80%
  7. Token: ' seemed' | Probability: 1.74%
  8. Token: ' ran' | Probability: 1.60%
  9. Token: ' would' | Probability: 1.48%
  10. Token: ' looked' | Probability: 1.44%
